# Import

In [1]:
import csv
import os
import pickle
import sys

import numpy as np
import pandas as pd
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
from gensim.corpora.dictionary import Dictionary
from gensim.models.ldamodel import LdaModel
from smart_open import open
from tqdm import tqdm

/opt/conda/lib/python3.10/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib and slated for removal in Python 3.12; see the module's documentation for alternative uses
  from imp import reload
/opt/conda/lib/python3.10/site-packages/gensim/matutils.py:22: DeprecationWarning: Please use `triu` from the `scipy.linalg` namespace, the `scipy.linalg.special_matrices` namespace is deprecated.
  from scipy.linalg.special_matrices import triu


## Add configuration file

In [2]:
sys.path.append("/home/jovyan/core/config/")

In [3]:
from ALL import config 

## Set condition

In [4]:
tqdm.pandas()
pd.set_option("display.max_columns", 100)
pd.set_option("display.max_rows", 50)

In [5]:
data_type="AgNewsTitle"

# Read data

In [6]:
df = pd.read_csv(
    f"../../Preprocessing/data/{data_type}/master.csv", index_col=0
)

In [7]:
with open(f"../../Preprocessing/data/{data_type}/class.csv", mode="r") as f:
    reader = csv.reader(f)
    class_labels = [label for label in reader]

In [8]:
model_nums = config["clustering"]["LDA"]["max_model_num"]

In [17]:
n_conmponents=config["data"][data_type]["class_num"]

# LDA

In [9]:
class Corpus:
    def __init__(self, path=None, texts=None):
        self.path = path
        self.texts = texts
        self.dictionary = Dictionary(texts)
        self.dictionary.filter_extremes()

    def __iter__(self):
        if self.path is not None:
            for line in open(self.path):
                # assume there's one document per line, tokens separated by whitespace
                yield self.dictionary.doc2bow(line.lower().split())
        else:
            for line in self.texts:
                yield self.dictionary.doc2bow(line)
                
    def __len__(self):
        return len(self.texts)

In [10]:
texts = df.words_nonstop.progress_apply(lambda x: x.split(' ') if x is not np.nan else [""]).tolist()
corpus = Corpus(texts=texts)
dictionary = Dictionary(texts)
dictionary.filter_extremes()

100%|██████████| 120000/120000 [00:00<00:00, 336633.89it/s]


In [11]:
os.makedirs(os.path.dirname(f"../data/{data_type}/LDA/"), exist_ok=True)
pickle.dump(dictionary, open(f"../data/{data_type}/LDA/dictionary.sav", "wb"))
pickle.dump(corpus, open(f"../data/{data_type}/LDA/corpus.sav", "wb"))

In [12]:
label = df["class"].to_numpy()

In [15]:
def getLDA(corpus,dictionary, n_components, seed, path):
    lda = LdaModel(
        corpus=corpus,
        id2word=dictionary,
        num_topics=n_components,
        alpha="auto",
        eval_every=5,
        random_state=seed,
    )
    # save model
    os.makedirs(os.path.dirname(path), exist_ok=True)
    lda.save(path)
    pred = [lda[docBow] for docBow in corpus]
    return pred, lda

In [ ]:
models_path = f"../data/{data_type}/LDA/model/"
pred_path = f"../data/{data_type}/LDA/pred/"

for model_num in tqdm(range(model_nums)):
    prob, lda = getLDA(
        corpus=corpus,
        dictionary=dictionary,
        n_components=n_conmponents,
        seed=model_num,
        path=f"{models_path}{model_num}"
    )
#     save prediction
    probDf = pd.DataFrame([dict(row) for row in prob]).fillna(0)
    pred = probDf.idxmax(axis=1).to_numpy()
    os.makedirs(f"{pred_path}", exist_ok=True)
    np.save(f"{pred_path}{model_num}.npy", pred)

  7%|▋         | 2/30 [01:28<20:44, 44.43s/it]

In [ ]:
pyLDAvis.enable_notebook()

vis = gensimvis.prepare(
      lda, corpus, dictionary, n_jobs = 1, sort_topics = False
      )

pyLDAvis.display(vis)